In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import  train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [10]:
(X_train, y_train), (X_valid, y_valid) = tf.keras.datasets.fashion_mnist.load_data()

def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))


model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(lr=1e-3),
                metrics=["accuracy"])

history = model_A.fit(X_train_A, y_train_A, epochs=20,
                    validation_data=(X_valid_A, y_valid_A))


model_A.save("my_model_A.h5")

Epoch 1/20
1500/1500 [==============================] - 4s 3ms/step - loss: 0.9870 - accuracy: 0.8286 - val_loss: 0.4040 - val_accuracy: 0.8630
Epoch 2/20
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3303 - accuracy: 0.8835 - val_loss: 0.3321 - val_accuracy: 0.8825
Epoch 3/20
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2919 - accuracy: 0.8975 - val_loss: 0.3068 - val_accuracy: 0.8953
Epoch 4/20
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2683 - accuracy: 0.9053 - val_loss: 0.2931 - val_accuracy: 0.8947
Epoch 5/20
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2514 - accuracy: 0.9106 - val_loss: 0.3142 - val_accuracy: 0.8889
Epoch 6/20
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2398 - accuracy: 0.9154 - val_loss: 0.2796 - val_accuracy: 0.9030
Epoch 7/20
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2291 - accuracy: 0.9189 - val_loss: 0.2858 - val_accuracy:

In [14]:
model_A = keras.models.load_model("my_model_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation='sigmoid'))
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

for layer in model_B_on_A.layers[:-1]:
  layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy", optimizer='sgd', metrics='acc')

history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16, validation_data=(X_valid_B, y_valid_B))

Epoch 1/16
7/7 [==============================] - 0s 36ms/step - loss: 0.7067 - acc: 0.7350 - val_loss: 0.2164 - val_acc: 0.9190
Epoch 2/16
7/7 [==============================] - 0s 20ms/step - loss: 0.2242 - acc: 0.9300 - val_loss: 0.1766 - val_acc: 0.9320
Epoch 3/16
7/7 [==============================] - 0s 20ms/step - loss: 0.1841 - acc: 0.9600 - val_loss: 0.1325 - val_acc: 0.9515
Epoch 4/16
7/7 [==============================] - 0s 19ms/step - loss: 0.1704 - acc: 0.9700 - val_loss: 0.1201 - val_acc: 0.9540
Epoch 5/16
7/7 [==============================] - 0s 19ms/step - loss: 0.1626 - acc: 0.9750 - val_loss: 0.1067 - val_acc: 0.9565
Epoch 6/16
7/7 [==============================] - 0s 19ms/step - loss: 0.1603 - acc: 0.9750 - val_loss: 0.1070 - val_acc: 0.9565
Epoch 7/16
7/7 [==============================] - 0s 19ms/step - loss: 0.1536 - acc: 0.9750 - val_loss: 0.1014 - val_acc: 0.9580
Epoch 8/16
7/7 [==============================] - 0s 19ms/step - loss: 0.1502 - acc: 0.9750 - val